# Kong Gateway 3.10 / Langchain agent

This notebook demonstrates how to use the AI Proxy plugin with Langchain agents.

## Prerequisites

Before running the notebook, ensure you have the following:

1. **Kong API Gateway running**: You will need a running instance of Kong API Gateway
2. **AI Proxy plugin configured**: An example is provided in this repo in the form of a deck dump.
3. **A [Tavily](https://tavily.com/) key** (free tier available). This is used to allow web searches for updated information.
This can either be entered interactively or added to the notebook secrets.

In [ ]:
!pip install -U langchain-openai numexpr langchain-community tavily-python langchain-openai langchainhub

In [54]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import initialize_agent,Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.chains import LLMMathChain
from google.colab import userdata
import os
import warnings

In [55]:
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

In [56]:
warnings.filterwarnings("ignore")

In [57]:
try:
  apikey=userdata.get('TAVILY_API_KEY')
except:
  raise ValueError('Please add your Tavily API key to the notebook secrets, ref: https://tavily.com/')

In [58]:
if 'host' not in globals():
  host = input("Enter your Gateway host url (http://localhost:8000/agent): ") or "http://localhost:8000/agent"

In [59]:
llm = AzureChatOpenAI(
    azure_endpoint=host,
    api_version="2023-03-15-preview",
    temperature=0,
    timeout=None,
    max_retries=3,
    model="gpt-4o",
    api_key="123",
    streaming=True
)

In [60]:
tavily_tool = TavilySearchResults(max_results=1)

In [61]:
math_tool = LLMMathChain(llm=llm)

In [62]:
prompt_template = """
You are an assistant that has access to two tools: a calculator and a web search tool. You can use the calculator to perform calculations or use the web search tool to find information.

here is the user input: {input}
"""

In [63]:
tools = [
    Tool(
        name="Tavily Search",
        func=tavily_tool.invoke,
        description="Search the web for information",
    ),
    Tool(
        name="Math Tool",
        func=math_tool.run,
        description="Perform calculations",
    )

]

In [64]:
agent = initialize_agent(llm=llm,tools=tools,prompt_template=prompt_template,verbose=True)

In [65]:
user_input = """Add 2 + the current population of the United States as of today, then tell me what 10% of the answer is. You will then multiply that percentage
by 100 and divide the result by 0.1. Finally, you will tell me if this is greater or less than the current population of the United Kingdom as of today."""

In [ ]:
response = agent.run(user_input)